# SQS: Code-First Pizza Ordering Lab

Provision an Amazon SQS Queue using Boto3, produce pizza order messages, consume them with a simulated worker, and explore distributed systems concepts like **Visibility Timeouts** and **Temporal Decoupling**.

## Setup — Get Your AWS Credentials

You need two values from your local machine to paste into **Colab Secrets**.

### Find your credentials
Open a terminal and run:
```
cat ~/.aws/credentials
```
You should see:
```
[default]
aws_access_key_id = AKIA...
aws_secret_access_key = Y+Co...
```

### Add secrets to Colab
1. Click the **🔑 key icon** in the left sidebar
2. Click **+ Add new secret** and create these two:
   - `AWS_ACCESS_KEY_ID` — starts with `AKIA...` (**required**)
   - `AWS_SECRET_ACCESS_KEY` — long string of letters/numbers (**required**)
3. Toggle **Notebook access** ON for each secret
4. Run the cell below — it should print `Connected as: arn:aws:sts::...`

In [ ]:
!pip install -q boto3

import boto3, os, json, time, uuid
from botocore.exceptions import ClientError

try:
    from google.colab import userdata
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
    print('Loaded credentials from Colab Secrets')
except (ImportError, KeyError):
    print('Not in Colab or secrets missing - using default AWS credential chain')

sts = boto3.client('sts', region_name='us-east-1')
identity = sts.get_caller_identity()
print(f"Connected as: {identity['Arn']}")

REGION = 'us-east-1'
QUEUE_NAME = f'pizza-order-queue-{uuid.uuid4().hex[:8]}'
sqs = boto3.client('sqs', region_name=REGION)
print(f'SQS Client Ready (Target Queue: {QUEUE_NAME})')

## Step 1: Provision the Queue
We will create a **Standard Queue**. By default, SQS guarantees **At-Least-Once** delivery.

The most important attribute here is **VisibilityTimeout**. This is how long SQS hides a message from others while a worker is processing it.

In [ ]:
try:
    response = sqs.create_queue(
        QueueName=QUEUE_NAME,
        Attributes={
            'VisibilityTimeout': '30',  # 30 seconds for workers to cook
            'MessageRetentionPeriod': '3600' # 1 hour (plenty for a lab)
        }
    )
    QUEUE_URL = response['QueueUrl']
    print(f'Queue Created Successfully!')
    print(f'URL: {QUEUE_URL}')
except ClientError as e:
    print(f'Error: {e}')

## Step 2: The Producer (Sending Orders)
Simulate a high-traffic pizza shop website. We send orders as JSON documents.
In a real system, this would be your API responding to users in milliseconds.

In [ ]:
orders = [
    {'id': '101', 'customer': 'Alice', 'pizza': 'Pepperoni', 'size': 'Large'},
    {'id': '102', 'customer': 'Bob', 'pizza': 'Veggie', 'size': 'Medium'},
    {'id': '103', 'customer': 'Carol', 'pizza': 'Hawaiian', 'size': 'Small'},
    {'id': '104', 'customer': 'David', 'pizza': 'Margherita', 'size': 'Large'}
]

print(f"Sending {len(orders)} orders to the queue...")

for order in orders:
    sqs.send_message(
        QueueUrl=QUEUE_URL,
        MessageBody=json.dumps(order),
        MessageAttributes={
            'Priority': {
                'DataType': 'String',
                'StringValue': 'High' if order['size'] == 'Large' else 'Normal'
            }
        }
    )
    print(f"  Order {order['id']} for {order['customer']} SENT.")

## Step 3: The Consumer (The Kitchen Worker)
The kitchen worker (consumer) pulls orders from the queue, "cooks" them (simulated by `time.sleep`), and then **deletes** the message.

**CRITICAL:** If you don't delete the message, SQS assumes the worker crashed and will put it back in the queue after the Visibility Timeout expires!

In [ ]:
def process_orders():
    print("Kitchen is open! Waiting for orders...")
    
    # Poll for messages
    response = sqs.receive_message(
        QueueUrl=QUEUE_URL,
        MaxNumberOfMessages=5,
        WaitTimeSeconds=10 # Long Polling
    )
    
    messages = response.get('Messages', [])
    if not messages:
        print("No orders in queue.")
        return

    for msg in messages:
        order = json.loads(msg['Body'])
        print(f"\n[WORKER] Picking up Order {order['id']} for {order['customer']}")
        
        # Simulate work
        print(f"  Cooking a {order['size']} {order['pizza']}...")
        time.sleep(3)
        
        # DELETE after success
        sqs.delete_message(
            QueueUrl=QUEUE_URL,
            ReceiptHandle=msg['ReceiptHandle']
        )
        print(f"  Order {order['id']} COMPLETE and DELETED from queue.")

process_orders()

## Step 4: Visibility Timeout Demo (Chaos Engineering)
Let's see what happens when the worker is slower than the timeout.

1. We set a **SHORT** timeout (5 seconds)
2. We make the worker **SLOW** (10 seconds)
3. We watch the message reappear in the queue while the worker is still "cooking"!

In [ ]:
# 1. Send one specific 'Chaos Pizza'
sqs.send_message(
    QueueUrl=QUEUE_URL,
    MessageBody=json.dumps({'id': 'CHAOS-1', 'customer': 'ChaosUser', 'pizza': 'Ghost Pepper'})
)

print("Chaos Order Sent. Overriding Visibility Timeout to 5 seconds...")

# 2. Poll with a short override
response = sqs.receive_message(
    QueueUrl=QUEUE_URL,
    MaxNumberOfMessages=1,
    VisibilityTimeout=5  # SHORTER than the work time below
)

if response.get('Messages'):
    msg = response['Messages'][0]
    print(f"\n[WORKER] Started cooking CHAOS pizza...")
    
    # Wait longer than 5 seconds
    for i in range(8):
        print(f"  Cooking... {i+1}s")
        time.sleep(1)
        
    print("\n[ALERT] While worker was cooking, the timeout expired!")
    print("Check the AWS Console: The message is visible again!")
    
    # Clean up
    sqs.delete_message(QueueUrl=QUEUE_URL, ReceiptHandle=msg['ReceiptHandle'])
    print("Cleaned up chaos message.")

## Step 5: Cleanup
Always delete your infrastructure when finished.

In [ ]:
print(f"Deleting queue {QUEUE_NAME}...")
sqs.delete_queue(QueueUrl=QUEUE_URL)
print("Queue Deleted.")

## Mini Challenge
1. Modify the Producer to send **10 orders** with a random "Pizza Type".
2. Modify the Consumer to skip orders that have a `Priority` of 'Normal' (don't delete them, just let them time out).
3. Run the worker and observe which orders stay in the queue and which are deleted.
4. **Screenshot your results** of the partial processing for your submission.